In [ ]:
# Teste de joguinho de produto escalar
# 12/8/22 - Murilo LCN

from threading import Thread
import time
import random
from IPython.display import clear_output

# Classezinha pra deixar organizado
class Dificuldade:
    def __init__(self, nome: str, pts: int, tempo: int, dimensao: int, intervalo: int):
        self.pontosMinimos: int = pts
        self.nome: str = nome
        self.tempo: int = tempo
        self.dimensao: int = dimensao
        self.intervalo: int = intervalo

# Função que calcula o produto escalar de dois vetores
def produtoEscalar(v1: list, v2: list) -> float:
    
    if len(v1) != len(v2):
        print("Não existe produto escalar de vetores de dimensões diferentes")
        return 0
    
    soma: float = 0
        
    for i in range(len(v1)):
        soma += v1[i] * v2[i]
    
    return soma

# Função que cria um vetor de dimensão N com números aleatórios entre [-intervalo, intervalo]
def pegarAleatorio(intervalo: int, n: int) -> list:
    v: list = []
    for i in range(n):
        v.append(random.randint(-intervalo, intervalo))
    return v

# Resposta dada pelo usuário
resposta: float = 0
    
# Flag: Se o timer chegou a zero
acabouTempo: bool = False

# Flag: Se foi recebido um input (i.e, a função perguntarResposta() terminou)
recebeuInput: bool = False

# Flag: Usado para rodar o jogo uma vez antes de perguntar se quer jogar denovo
temp: bool = False

# Variáveis de jogo
timeout: int = 10  # Tempo de cada rodada
vezesJogadas: int = 0
vezesGanhas: int = 0

dificuldadeAtual: Dificuldade = None
escolha: int = 0

# Usado para ir mudando as dificuldades conforme acertos
count: int = 0

dificuldades = [
    # Nome, pts, tempo, dimensao, intervalo
    Dificuldade("Estudante de humanas", 0, 20, 1, 3),
    Dificuldade("Nutella", 3, 20, 1, 4),
    Dificuldade("Fácil", 6, 20, 2, 5),
    Dificuldade("Médio", 9, 20, 2, 7),
    Dificuldade("Difícil", 12, 25, 3, 10),
    Dificuldade("Bem difícil", 15, 30, 3, 12),
    Dificuldade("Fodão", 18, 25, 4, 15),
    Dificuldade("Calculadora humana", 21, 20, 5, 20),
    Dificuldade("'Vocês viram isso no ensino médio'", 24, 15, 6, 20)
]
    

def perguntarResposta():
    global recebeuInput
    
    try:
        resposta = float(input("Insira o produto escalar: "))
    except ValueError:
        resposta = None
    recebeuInput = True
    return resposta


# Função a ser rodada em thread que faz a contagem
def iniciarContagem(tempo: int):
    global recebeuInput
    global acabouTempo

    print("Iniciando contagem! ")
    
    while (tempo > 0):
        # Se recebeu input, para de esperar
        if (recebeuInput == True):
            print("\nRecebido input\n")
            break
        
        # Printa o tempo restante, espera 1s e diminui 1s do tempo restante
        print(f"{tempo}s", end=" ")
        time.sleep(1)
        tempo -= 1
    
    if (not recebeuInput):
        print("\nAcabou o tempo!")
        print("Insira qualquer valor para continuar") 
        # O usuário não colocou nenhum input a tempo
        acabouTempo = True
    
    # Sai da thread
    return
    

print("Escolha sua dificuldade\n")
for i, diff in enumerate(dificuldades):
    print(f">> {i} {diff.nome}")

# Pega a dificuldade escolhida
try:
    escolha = int(input(">> "))
    dificuldadeAtual = dificuldades[escolha]
    
    if escolha > len(dificuldades) - 1 or escolha < 0:
        raise IndexError
        
except (ValueError, IndexError):
    escolha = 0
    dificuldadeAtual = dificuldades[escolha]
    
timeout = dificuldadeAtual.tempo
count = dificuldadeAtual.pontosMinimos    

clear_output(wait=False)
    
# Loop principal
while True:
    
    # Vê se os pontos já são suficientes para passar de fase (ou regredir para outra mais fácil)
    # Nota: Ficou meio feio mas isso faz com que se criar ou tirar dificuldades não precisa mexer em mais nada
    try:
        if count > dificuldades[escolha + 1].pontosMinimos and escolha != len(dificuldades) - 1:
            dificuldadeAtual = dificuldades[escolha + 1]
            escolha = escolha + 1
    except IndexError:
        pass
    try:
        if count <= dificuldades[escolha - 1].pontosMinimos and escolha != 0:
            dificuldadeAtual = dificuldades[escolha - 1]
            escolha = escolha - 1
    except IndexError:
        pass
    
        
    print(f"Dificuldade do próximo jogo: {dificuldadeAtual.nome}")
    
    # Se não é a primeira vez jogando
    if (temp):
        try:
            i = int(input("Escolha uma opção:\n[0] >> Sair\n[1] >> Jogar novamente\n"))
            if (i == 0):
                print("Até a próxima")
                print(f"Você ganhou {vezesGanhas} partidas de {vezesJogadas}")
                break
        except ValueError:
                pass
    
        clear_output(wait=False)
    
    u = pegarAleatorio(dificuldadeAtual.intervalo, dificuldadeAtual.dimensao)
    v = pegarAleatorio(dificuldadeAtual.intervalo, dificuldadeAtual.dimensao)
    
    res = produtoEscalar(u, v)
    
    print(f"Calcule o produto escalar de: \nU = {u}\nV = {v}")
    
    # Thread alternativa para fazer a contagem
    # Thread principal ficará para pegar o input
    t1 = Thread(target=iniciarContagem, args=[timeout])
    t1.start()
    resposta = perguntarResposta()
    
    # Faz o sistema esperar a contagem terminar antes de prosseguir no loop
    t1.join()
    if (resposta == res and not acabouTempo):
        print("Parabéns, tu é brabo mesmo")
        count += 1
        vezesGanhas += 1
    elif (acabouTempo):
        print("Demorou demais")
        if count != 0:
            count -= 1
    else:
        print("Resposta errada")
        if count != 0:
            count -= 1
    
    print(f"A resposta era {res}")
    
    recebeuInput = False
    acabouTempo = False
    temp = True
    
    vezesJogadas += 1